In [17]:
import time
import numpy as np

#### 유틸리티 함수

In [18]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

#### 뉴런 구현

In [23]:
class Neuron:
    def __init__(self, W, b, a):
        # Model Parameter
        self.W = W
        self.b = b
        self.a = a
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        
    def __call__(self, x):
        return self.a(_m(_t(self.W), x) + self.b) # activation((W^T)x + b)

#### 심층신경망 구현

In [24]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i,o)), np.zeros((o,))
        
        self.sequence = list()
        # First hidden layer
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))

        # Hidden layers
        for _ in range(hidden_depth - 1):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Neuron(W, b, activation))
    
    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x
    
    def calc_gradient(self, x, y, loss_func):
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def eval_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y)
        
        for layer_id, layer in enumerate(self.sequence): # iterate layer
            for w_i, w in enumerate(layer.W): # iterate W (row)
                for w_j, ww in enumerate(w): # iterate W (col)
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon
                    
                    new_neuron = Neuron(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    grad = (loss_func(h, y) - loss) / epsilon # (f(x_eps) - f(x)) / epsilon
                    layer.dW[w_i][w_j] = grad
                    
                for b_i, bb in enumerate(layer.b):
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon
                    
                    new_neuron = Neuron(layer.W, b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    grad = (loss_func(h, y) - loss) / epsilon # (f(x_eps) - f(x)) / epsilon
                    layer.db[b_i] = grad
    
        return loss

#### 경사하강 학습법

In [25]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

### Training

In [26]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5,
          num_neuron=32,
          num_input=10,
          num_output=2,
          activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {} : Test loss {}'.format(epoch, loss))
print('{} seconds elapsed'.format(time.time() - t))

Epoch 0 : Test loss 0.5002245382277205
Epoch 1 : Test loss 0.49740958818180775
Epoch 2 : Test loss 0.4946110321080279
Epoch 3 : Test loss 0.49182933791765077
Epoch 4 : Test loss 0.4890649553961113
Epoch 5 : Test loss 0.4863183159515946
Epoch 6 : Test loss 0.4835898324103413
Epoch 7 : Test loss 0.4808798988578826
Epoch 8 : Test loss 0.47818889052517766
Epoch 9 : Test loss 0.4755171637189259
Epoch 10 : Test loss 0.47286505579438787
Epoch 11 : Test loss 0.4702328851696852
Epoch 12 : Test loss 0.4676209513800118
Epoch 13 : Test loss 0.465029535169881
Epoch 14 : Test loss 0.46245889862223766
Epoch 15 : Test loss 0.45990928532212516
Epoch 16 : Test loss 0.4573809205532417
Epoch 17 : Test loss 0.45487401152571527
Epoch 18 : Test loss 0.45238874763263504
Epoch 19 : Test loss 0.4499253007339018
Epoch 20 : Test loss 0.4474838254650361
Epoch 21 : Test loss 0.44506445956894175
Epoch 22 : Test loss 0.442667324249003
Epoch 23 : Test loss 0.44029252454097056
Epoch 24 : Test loss 0.437940149702182
Epo